In [1]:
from scipy import stats

In [2]:
from scipy.stats import spearmanr

In [3]:
from scipy.stats import ConstantInputWarning

In [4]:
from src.pipeline.evaluation.evaluate_regression import get_rankings, average_spearman, custom_train_test_split

In [5]:
import pandas as pd
import os

from category_encoders import OneHotEncoder
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor

from src.configuration import load_dataset

factors = ["dataset", "model", "tuning", "scoring"]
new_index = "encoder"

# ---- load data...
DATA_DIR = Path( "../data/raw")

df_train = load_dataset(DATA_DIR / "dataset_train.csv")
X_train, X_test, y_train, y_test = custom_train_test_split(df_train, factors, "cv_score")

# ---- predict
dummy_pipe = Pipeline([("encoder", OneHotEncoder()), ("model", DecisionTreeRegressor())])
y_pred = pd.Series(dummy_pipe.fit(X_train, y_train).predict(X_test), index=y_test.index, name="cv_score_pred")
df_pred = pd.concat([X_test, y_test, y_pred], axis=1)

# ---- convert to rankings and evaluate
rankings_test = get_rankings(df_pred, factors=factors, new_index=new_index, target="cv_score")
rankings_pred = get_rankings(df_pred, factors=factors, new_index=new_index, target="cv_score_pred")
print(average_spearman(rankings_test, rankings_pred))

0.49130677830306363
